In [ ]:
from clickhouse_connect import get_client
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

client = get_client(
    host="xx",
    port=xxx,
    username="xxx",
    password="xxx"
)

In [ ]:
def get_all_cancel_orders(date: str = '2025-03-03'):
    """
    获取指定日期所有撤单的编号
    """
    sql = f"""
    SELECT 
        code,
        date_time,
        trade_flag,
        exg,
        bidno,  -- 撤买单的原始编号
        askno   -- 撤卖单的原始编号
    FROM stock_base.zb 
    WHERE date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:20:00'  -- 撤单只在09:20前
        AND trade_flag = '4'  -- 撤单标识
        AND (bidno IS NOT NULL OR askno IS NOT NULL)
        AND exg = '1'
    """
    
    df = client.query_df(sql)
    cancel_ids = []
    for _, row in df.iterrows():
        if pd.notna(row['bidno']) and int(row['bidno']) > 0:
            cancel_ids.append(int(row['bidno']))  # 撤买单编号
        elif pd.notna(row['askno']) and int(row['askno']) > 0:
            cancel_ids.append(int(row['askno']))  # 撤卖单编号

    return df, cancel_ids

# 使用
cancel_df, cancel_ids = get_all_cancel_orders('2025-03-03')
print(f"\n撤单编号示例: {cancel_ids[:10]}")  # 前10个

In [ ]:
def get_all_data(code: str, date: str = '2025-03-03', cancel_ids: list = []):
    """
    获取股票数据，并排除被撤掉的订单
    cancel_ids: 从撤单表获取的所有撤单编号列表
    """
    # 基础查询
    sql = f"""
    SELECT 
        time_int,
        price,
        volume,
        side,
        ordno,
        type,
        trade_flag,
        exg
    FROM stock_base.zb 
    WHERE code = '{code}'
        AND date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:25:00'
        AND type IS NOT NULL  -- 挂单数据
        AND price IS NOT NULL
        AND volume IS NOT NULL
        AND exg = '1'
    ORDER BY time_int
    """
    
    df = client.query_df(sql)
    
    if df.empty:
        return df
    
    # 在Python中过滤：排除被撤掉的订单
    if cancel_ids:
        # 过滤掉ordno在cancel_ids中的订单
        # 注意：ordno可能是空值，先处理
        mask = ~df['ordno'].isin(cancel_ids)
        df = df[mask].copy()
    
    return df

In [ ]:
def get_all_codes(date: str = '2025-03-03'):
    sql = f"""
    SELECT DISTINCT code
    FROM stock_base.zb 
    WHERE date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:25:00'
        AND type IS NOT NULL
        AND price IS NOT NULL
        AND exg = '1'
    LIMIT 200
    """
    
    try:
        df = client.query_df(sql)
        return df['code'].tolist() if not df.empty else []
    except Exception as e:
        print(f"查询失败: {e}")
        return []

In [ ]:
def calculate_opening_price(buy_orders, sell_orders, prev_close):
    """
    完整的开盘价计算，包含规则三
    """
    if prev_close is None:
        prev_close = 0
    
    # 1. 获取所有价格
    all_prices = sorted(set(
        list(buy_orders['price'].unique()) + 
        list(sell_orders['price'].unique())
    ))
    
    max_volume = 0
    candidate_prices = []
    
    # 2. 找出所有可能价格中满足条件三的最大成交量
    for price in all_prices:
        # 买方累计：价格>=当前价
        buy_total = buy_orders[buy_orders['price'] >= price]['volume'].sum()
        # 卖方累计：价格<=当前价
        sell_total = sell_orders[sell_orders['price'] <= price]['volume'].sum()
        volume = min(buy_total, sell_total)
        
        if volume == 0:
            continue
        
        # 检查规则三：该价格相同的买方或卖方至少有一方全部成交
        buy_at_price = buy_orders[buy_orders['price'] == price]['volume'].sum()
        sell_at_price = sell_orders[sell_orders['price'] == price]['volume'].sum()
        
        # 条件三检查
        condition_three = False
        if buy_at_price > 0 and sell_at_price > 0:
            # 双方都有挂单，必须至少一方全部成交
            # 买单能成交多少（在总成交量内）
            buy_can_trade = min(buy_at_price, volume)
            # 卖单能成交多少
            sell_can_trade = min(sell_at_price, volume)
            
            condition_three = (buy_can_trade == buy_at_price) or (sell_can_trade == sell_at_price)
        else:
            # 只有一方有挂单，自动满足
            condition_three = True
        
        if not condition_three:
            continue  # 不满足规则三，跳过
        
        # 更新最大成交量
        if volume > max_volume:
            max_volume = volume
            candidate_prices = [price]
        elif volume == max_volume:
            candidate_prices.append(price)
    
    if not candidate_prices:
        return prev_close if prev_close > 0 else 0, 0
    
    # 3. 只有一个价格
    if len(candidate_prices) == 1:
        return candidate_prices[0], max_volume
    
    # 4. 多个价格：买卖量差最小
    min_diff = float('inf')
    second_candidates = []
    
    for price in candidate_prices:
        buy_above = buy_orders[buy_orders['price'] >= price]['volume'].sum()
        sell_below = sell_orders[sell_orders['price'] <= price]['volume'].sum()
        diff = abs(buy_above - sell_below)
        
        if diff < min_diff:
            min_diff = diff
            second_candidates = [price]
        elif diff == min_diff:
            second_candidates.append(price)
    
    if len(second_candidates) == 1:
        return second_candidates[0], max_volume
    
    # 5. 取最接近前收盘价
    if prev_close == 0:
        return second_candidates[len(second_candidates)//2], max_volume
    
    closest_price = min(second_candidates, key=lambda p: abs(p - prev_close))
    return closest_price, max_volume

In [ ]:
# 从日线表获取真实开盘价
def get_real_opening_price(code, date:str = '2025-03-03'):
    sql = f"""
    SELECT 
        open,
        pclose 
    FROM stock_base.daily 
    WHERE code = '{code}' AND date = '{date}'
    LIMIT 1
    """
    
    try:
        df = client.query_df(sql)
        if not df.empty:
            return float(df.iloc[0]['open']),float(df.iloc[0]['pclose'])
    except Exception as e:
        print(f"查询失败: {e}")
    
    return None

In [ ]:
cancel_df, cancel_ids = get_all_cancel_orders('2025-03-03')
codes = np.unique(get_all_codes(date='2025-03-03'))
all_codes = len(codes)
print(f"共有{all_codes}支股票")
count = 0
wrong_code = {}

for code in codes:
    df = get_all_data(code,cancel_ids=cancel_ids)
    buy_orders = df[df['side'] == b'B'].copy()
    sell_orders = df[df['side'] == b'S'].copy()
    real_price, prev_close = get_real_opening_price(code)

    best_price, max_volume = calculate_opening_price(buy_orders,sell_orders,prev_close)
    
    
    print(f"计算:{best_price}     实际:{real_price}")
    if(real_price == best_price):
        count+=1
    else:
        wrong_code[code] = (best_price, real_price)

print(f"计算正确{count}支股票")
print(wrong_code)